In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from aust_covid.inputs import load_google_mob_year_df
PROJECT_PATH = Path().resolve().parent
DATA_PATH = PROJECT_PATH / 'data'

In [ ]:
locations = [
    'school', 
    'home', 
    'work', 
    'other_locations',
]

In [ ]:
raw_location_matrices = {i: pd.read_csv(DATA_PATH / f'{i}.csv', index_col=0).to_numpy() for i in locations}

In [ ]:
raw_data = pd.concat(
    [
        pd.read_csv(DATA_PATH / '2021_AU_Region_Mobility_Report.csv', index_col=8),
        pd.read_csv(DATA_PATH / '2022_AU_Region_Mobility_Report.csv', index_col=8),
    ]
)
state_data = raw_data.loc[raw_data['sub_region_1'].notnull() & raw_data['sub_region_2'].isnull()]
wa_data = state_data.loc[state_data['sub_region_1'] == 'Western Australia']
non_wa_data = state_data.loc[state_data['sub_region_1'] != 'Western Australia']
mob_cols = [c for c in wa_data.columns if '_percent_change_from_baseline' in c]
wa_mobility = wa_data[mob_cols]
non_wa_mobility = non_wa_data[['sub_region_1'] + mob_cols]
wa_mobility.columns = [c.replace('_percent_change_from_baseline', '') for c in wa_mobility.columns]
non_wa_mobility.columns = [c.replace('_percent_change_from_baseline', '') for c in non_wa_mobility.columns]

In [ ]:
non_wa_mobility

In [ ]:
something = non_wa_mobility.groupby(non_wa_mobility.index)

In [ ]:
non_wa_mobility['fake_weights'] = 1.0

In [ ]:
non_wa_mobility

In [ ]:
something = non_wa_mobility.groupby(non_wa_mobility.index).apply(lambda x: np.average(x['retail_and_recreation'], weights=x['fake_weights']))

In [ ]:
rel_mob_df = 1.0 + mob_df * 1e-2